# SOCRAT - SPACEDOS Decoder

In [1]:
import glob

In [2]:
#h = b'\x30\x00\xde' #HKSD 112B
#h = b'\x30\x00\xc6' #5xBESD 100B
h = b'\x30\x00\x2e' #DPSD 24B
#h = b'\x55\x00' #DPSD 24B

#h = b'\x00\x02\x00\x02\x00\x02\x00\x02\x00\x00\x00\x02'
#h = b'\x00\xde'
#h = b'\x02\x00\x06\x00'
#h = b'\x0a\x00\x04\x00'

print(h)
for fn in glob.iglob("data/**/**/*.SOKRAT_R"): # generator, search immediate subdirectories 
    #print(fn)
    with open(fn, 'rb') as f:
        s = f.read()
        found = s.find(h)
        if (found>0):
            print(fn, found)
            print( (s).hex())  
    
            next = 0
            for b in s:
                print(hex((b >> 1) | next)[2:] , ' ', end='' )
                if ((b & 1) == 1):
                    next = 0x80
                else:
                    next = 0
            print()
            print()

None

b'0\x00.'


# HKSD

In [3]:
h = b'\x30\x00\xde' #HKSD 112B

for fn in glob.iglob("data/**/**/*.SOKRAT_R"): # generator, search immediate subdirectories 
    #print(fn)
    with open(fn, 'rb') as f:
        s = f.read()
        found = s.find(h)
        if (found>0):
            print(fn, found)
            print( (s).hex())  
    
            next = 0
            value = []
            offset = 0
            for b in s:
                value.append((b >> 1) | next)
                print(hex(value[-1])[2:] , end=' ' )
                if ((b & 1) == 1):
                    next = 0x80
                else:
                    next = 0
            print()
            print()

            start = 23
            
            print('$HKSD',end=',')
            print(value[start]+value[start+1]*2**8,end=',')
            print(value[start+2]*2**32+value[start+3]*2**16+value[start+4]*2**8+value[start+5],end=',')
            for i in range(start+7,(start+7)+50*2,2):
                print(value[i]+value[i+1]*2**8,end=',')
            print()    
            print()    

#$HKSD,0,e,8003,ff,0,0,7ff4,3,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
None

data/3340B.2020.02.11.14.50.04/Report RS19S_Socrat/000_0299.SOKRAT_R 17
200211114136cccc57653d077ead5500743000de00000000000000001c000701fe0000000001e8fe0600020002000200020000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000fffffb8e7
10 1 8 88 a0 9b 66 66 2b b2 9e 83 bf 56 aa 80 3a 18 0 6f 0 0 0 0 0 0 0 0 e 0 3 80 ff 0 0 0 0 0 f4 7f 3 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 ff ff dc 73 

$HKSD,0,14,32771,255,0,0,32756,3,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

data/2132B.2019.11.23.16.14.13/Report RS19S_Socrat/000_0130.SOKRAT_R 17
191123130902cccc57653d077ead5500743000de00000000000000001c000701fe0000000001e8fe0600020002000200020000000200000000000000000000000000000000000000000000000

data/3340S/Report RS19S_Socrat/000_0345.SOKRAT_R 17
200211114138cccc57653d077ead5500743000de00000000000000001c000701fe0000000001e8fe0600020002000200020000000200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000fffffb8e7
10 1 8 88 a0 9c 66 66 2b b2 9e 83 bf 56 aa 80 3a 18 0 6f 0 0 0 0 0 0 0 0 e 0 3 80 ff 0 0 0 0 0 f4 7f 3 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 ff ff dc 73 

$HKSD,0,14,32771,255,0,0,32756,3,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

data/3340S/Report RS19S_Socrat/000_0344.SOKRAT_R 17
200211114137cccc57653e077ead5500743000de00000000000000001c000700fe0000000001e8fe06000200020002000200000002000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

c 84 14 8 28 1c e6 66 2b b2 9e 83 bf 56 aa 80 3a 18 0 6f 64 0 0 0 61 4e 0 0 60 ea 0 0 2b 4 0 0 0 0 0 0 3d 54 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3d 54 0 0 0 0 0 0 3d 54 0 0 1 0 0 0 ca e9 0 0 1 0 0 0 ca e9 0 0 0 0 0 0 3d 54 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 59 ff ff 88 e5 

$HKSD,24832,335007449184,0,1067,0,0,0,21565,0,0,0,0,0,0,0,21565,0,0,0,21565,0,1,0,59850,0,1,0,59850,0,0,0,21565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

data/769B/Report RS19S_Socrat/000_0007.SOKRAT_R 17
190828105053cccc57653d077ead5500743000dec8000000c29c0000c1d4000056080000000000007aa800000000000000000000000000007aa80000000000007aa800000200000195d200110200000195d20000000000007aa800000000000000000000000000000000000000000000000000000000000000000000000000000000000000b3ffff11ca
c 84 14 8 28 29 e6 66 2b b2 9e 83 bf 56 aa 80 3a 18 0 6f 64 0 0 0 61 4e 0 0 60 ea 0 0 2b 4 0 0 0 0 0 0 3d 54 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3d 54 0 0 0 0 0 0 3d 54 0 0 1 0 0 0 ca e9 0 8 81 0 0 0 ca e9 0 

data/2117A/Report RS19S_Socrat/000_0173.SOKRAT_R 17
191122131853cccc57653d077ead5500743000de000000000000060004000400020000000000000000000000020006000e000a0004000400000000000000000004000a00240016000c000400000000000000000006000c0030001c001a0006000000000000000000080010003a0024002000060002000000000000000a001a004200300000e6ffff9a01
c 88 91 9 8c 29 e6 66 2b b2 9e 83 bf 56 aa 80 3a 18 0 6f 0 0 0 0 0 0 3 0 2 0 2 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 3 0 7 0 5 0 2 0 2 0 0 0 0 0 0 0 0 0 2 0 5 0 12 0 b 0 6 0 2 0 0 0 0 0 0 0 0 0 3 0 6 0 18 0 e 0 d 0 3 0 0 0 0 0 0 0 0 0 4 0 8 0 1d 0 12 0 10 0 3 0 1 0 0 0 0 0 0 0 5 0 d 0 21 0 18 0 0 73 7f ff cd 0 

$HKSD,0,196610,2,1,0,0,0,0,0,1,3,7,5,2,2,0,0,0,0,2,5,18,11,6,2,0,0,0,0,3,6,24,14,13,3,0,0,0,0,4,8,29,18,16,3,1,0,0,0,5,13,33,

data/3356S/Report RS19S_Socrat/000_0366.SOKRAT_R 17
200212130722cccc57653d077ead5500743000de27f7bc278cc5d5965318889aa1a28100ff7e95777ecc76850dc5f1e32dc400875f9a1c7f228a3029f4ec73f403224f56e170a3c296f38632d0266006c6fd1a0a7dfbde2e91f7cf5ce0

# BESD

In [4]:
h = b'\x30\x00\xc6' #5xBESD 100B

for fn in glob.iglob("data/**/**/*.SOKRAT_R"): # generator, search immediate subdirectories 
    #print(fn)
    with open(fn, 'rb') as f:
        s = f.read()
        found = s.find(h)
        if (found>0):
            print(fn, found)
            print((s).hex())  
    
            next = 0
            value = []
            offset = 0
            for b in s:
                value.append((b >> 1) | next)
                print(hex(value[-1])[2:] , end=' ' )
                if ((b & 1) == 1):
                    next = 0x80
                else:
                    next = 0
            print()
            print()
           
            for n in range(23,120,20):
                
                print('$BESD',end=',')
                for i in range(n,n+18,2):
                    print(value[i]*2**8+value[i+1],end=',')
                print()  
            print()

#$BESD,0,3,2,2,1,0,0,0,0
None

data/3340B.2020.02.11.14.50.04/Report RS19S_Socrat/000_0296.SOKRAT_R 17
200211114046cccc57653c077ead5500683000c6000000000000060004000400020000000000000000000000020006000e000a0004000400000000000000000004000a00240016000c000400000000000000000006000c0030001c001a0006000000000000000000080010003a002400200006000200000000a0000000000000000000000000ffffe1bd
10 1 8 88 a0 23 66 66 2b b2 9e 3 bf 56 aa 80 34 18 0 63 0 0 0 0 0 0 3 0 2 0 2 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 3 0 7 0 5 0 2 0 2 0 0 0 0 0 0 0 0 0 2 0 5 0 12 0 b 0 6 0 2 0 0 0 0 0 0 0 0 0 3 0 6 0 18 0 e 0 d 0 3 0 0 0 0 0 0 0 0 0 4 0 8 0 1d 0 12 0 10 0 3 0 1 0 0 0 0 50 0 0 0 0 0 0 0 0 0 0 0 0 7f ff f0 de 

$BESD,0,3,2,2,1,0,0,0,0,
$BESD,1,3,7,5,2,2,0,0,0,
$BESD,2,5,18,11,6,2,0,0,0,
$BESD,3,6,24,14,13,3,0,0,0,
$BESD,4,8,29,18,16,3,1,0,0,

data/3340B.2020.02.11.14.50.04/Report RS19S_Socrat/000_0297.SOKRAT_R 17
200211114047cccc57653d077ead5500683000c6000000000000060004000400020000000000000000000000020006000e000a0004000400000000000000000004000a0024001

# DPSD

In [5]:
h = b'\x30\x00\x2e' #DPSD 24B

print(h)
for fn in glob.iglob("data/**/**/*.SOKRAT_R"): # generator, search immediate subdirectories 
    #print(fn)
    with open(fn, 'rb') as f:
        s = f.read()
        found = s.find(h)
        if (found>0):
            print(fn, found)
            print( (s).hex())  
    
            next = 0
            value = []
            offset = 0
            for b in s:
                value.append((b >> 1) | next)
                print(hex(value[-1])[2:] , end=' ' )
                if ((b & 1) == 1):
                    next = 0x80
                else:
                    next = 0
            print()
            print()
           
            for n in range(24,120,20):
                
                print('$BESD',end=',')
                for i in range(n,n+18,2):
                    print(value[i]+value[i+1]*2**8,end=',')
                print()  
            print()

None

b'0\x00.'
